In [ ]:
"""
PASS/FAIL PREDICTION
------------------------------------------------------
Goal:
  Learn a simple pattern:
    Given 4 inputs (Math, Science, English, Attendance),
    predict Pass(1) or Fail(0).

What this code demonstrates:
  1) Creating tensors for inputs/labels
  2) Building a Sequential Keras model (Input -> Dense -> Dense)
  3) Compiling the model (optimizer, loss, metrics)
  4) Training the model (fit)
  5) Making predictions on new inputs
  6) Checking shapes to avoid common mistakes

Note:
  This is a tiny toy dataset. It's meant for learning the workflow,
  not for real accuracy.
"""

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input


# ------------------------------------------------------
# STEP 1: Create a small, meaningful dataset
# ------------------------------------------------------
# Each row is ONE student:
# [MathScore, ScienceScore, EnglishScore, AttendancePercent]
X = tf.constant([
    [80, 85, 78, 90],   # likely Pass
    [60, 58, 55, 70],   # likely Fail
    [90, 92, 88, 95],   # likely Pass
    [50, 45, 48, 60],   # likely Fail
    [72, 75, 70, 80],   # likely Pass-ish
    [40, 42, 38, 50],   # likely Fail
], dtype=tf.float32)

# Labels (ground truth):
# 1 = Pass, 0 = Fail
# IMPORTANT: For binary classification, labels are typically shape (N, 1)
y = tf.constant([
    [1],
    [0],
    [1],
    [0],
    [1],
    [0],
], dtype=tf.float32)

print("STEP 1: Data shapes")
print("X shape:", X.shape)  # (num_samples, 4)
print("y shape:", y.shape)  # (num_samples, 1)
print("-" * 60)


# ------------------------------------------------------
# STEP 2: Build the model (Sequential = layers in a straight line)
# ------------------------------------------------------
# Model design:
#   Input(shape=(4,))         -> each sample has 4 features
#   Dense(8, relu)            -> learns patterns/feature combinations
#   Dense(1, sigmoid)         -> outputs probability of Pass (0 to 1)
model = Sequential([
    Input(shape=(4,)),                 # Explicit input layer (best practice)
    Dense(8, activation="relu"),       # Hidden layer
    Dense(1, activation="sigmoid")     # Output layer for binary classification
])

print("STEP 2: Model summary (shows shape flow)")
model.summary()
print("-" * 60)


# ------------------------------------------------------
# STEP 3: Compile the model
# ------------------------------------------------------
# optimizer: how weights are updated (Adam is a good default)
# loss: binary_crossentropy is used for binary classification (Pass/Fail)
# metrics: accuracy to see performance during training
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

print("STEP 3: Model compiled")
print("-" * 60)


# ------------------------------------------------------
# STEP 4: Train the model
# ------------------------------------------------------
# epochs: number of times the model sees the full dataset
# verbose=1 shows training progress; use verbose=0 to hide it
print("STEP 4: Training...")
history = model.fit(X, y, epochs=200, verbose=0)
print("Training complete.")
print("-" * 60)


# ------------------------------------------------------
# STEP 5: Evaluate on the same tiny dataset (just for demo)
# ------------------------------------------------------
loss, acc = model.evaluate(X, y, verbose=0)
print("STEP 5: Evaluation on training data")
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {acc:.4f}")
print("-" * 60)


# ------------------------------------------------------
# STEP 6: Make predictions on NEW students
# ------------------------------------------------------
# IMPORTANT: Input to model.predict must be 2D: (batch, features)
# So for ONE student, shape should be (1, 4)
new_students = tf.constant([
    [75, 80, 70, 85],   # likely Pass
    [55, 50, 52, 65],   # likely Fail/Borderline
], dtype=tf.float32)

print("STEP 6: Prediction input shape:", new_students.shape)

pred_probs = model.predict(new_students, verbose=0)  # probabilities between 0 and 1

# Convert probability to Pass/Fail using threshold 0.5
pred_labels = (pred_probs >= 0.5).astype(int)

for i, (prob, label) in enumerate(zip(pred_probs, pred_labels), start=1):
    print(f"Student {i}: pass_probability={prob[0]:.3f} -> predicted_label={label[0]}")

print("-" * 60)


# ------------------------------------------------------
# STEP 7 (Optional): Show learned weights (Dense layers store weights)
# ------------------------------------------------------
# This is optional but useful to show trainees that the model "learned numbers".
print("STEP 7 (Optional): Learned weights in each Dense layer")
for layer in model.layers:
    if isinstance(layer, Dense):
        W, b = layer.get_weights()
        print(f"\nLayer: {layer.name}")
        print("Weights shape:", W.shape)
        print("Bias shape:", b.shape)


In [ ]:
test_student = tf.constant(
    [[85, 88, 82, 92]],  # High scores & attendance → should PASS
    dtype=tf.float32
)

print("STEP 8: Testing with a single student")
print("Test student input shape:", test_student.shape)

# Get prediction (probability)
test_prediction = model.predict(test_student, verbose=0)

# Interpret prediction
pass_probability = test_prediction[0][0]
predicted_label = 1 if pass_probability >= 0.5 else 0

print(f"Predicted pass probability: {pass_probability:.3f}")
print("Predicted result:", "PASS ✅" if predicted_label == 1 else "FAIL ❌")
print("-" * 60)